In [1]:
import datalabframework as dlf

In [2]:
dlf.project.rootpath()

'/home/natbusa/Projects/dsp-titanic/src'

#### Init Spark

In [3]:
engine = dlf.engines.get('spark')
spark = engine.context()

In [4]:
#print out name and version
'{}:{}'.format(engine.info['context'], spark.sparkSession.version)

'spark:2.3.1'

In [5]:
df = engine.read('.etl.features.train')

In [6]:
df.show(truncate=False)

+---+-----+----------------------------------------------------------------------+
|id |label|features                                                              |
+---+-----+----------------------------------------------------------------------+
|246|0    |(19,[0,1,3,6,10,16],[44.0,90.0,1.0,1.0,1.0,1.0])                      |
|346|1    |(19,[0,1,4,10,17],[24.0,13.0,1.0,1.0,1.0])                            |
|360|1    |(19,[0,1,2,4,10],[27.046921317757153,7.8792,1.0,1.0,1.0])             |
|367|1    |(19,[0,1,3,5,10,18],[60.0,75.25,1.0,1.0,1.0,1.0])                     |
|476|0    |(19,[0,1,3,4,10,16,17],[42.24366964356571,52.0,1.0,1.0,1.0,1.0,1.0])  |
|539|0    |(19,[0,1,2,4,10,16,17],[28.93837380874933,14.5,1.0,1.0,1.0,1.0,1.0])  |
|599|0    |(19,[0,1,2,4,10,16,18],[27.140094702369655,7.225,1.0,1.0,1.0,1.0,1.0])|
|725|1    |(19,[0,1,3,5,10,16,17],[27.0,53.1,1.0,1.0,1.0,1.0,1.0])               |
|855|0    |(19,[0,1,5,10,17],[44.0,26.0,1.0,1.0,1.0])                            |
|861

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
pipeline = Pipeline(stages=[lr])

model = pipeline.fit(df)
prediction = model.transform(df)

In [8]:
prediction.columns

['id', 'label', 'features', 'rawPrediction', 'probability', 'prediction']

In [9]:
prediction.select('label', 'prediction','probability').show(truncate=False)

+-----+----------+-----------------------------------------+
|label|prediction|probability                              |
+-----+----------+-----------------------------------------+
|0    |0.0       |[0.6338514343795529,0.36614856562044706] |
|1    |1.0       |[0.16785546841435905,0.8321445315856411] |
|1    |1.0       |[0.32706095199330093,0.6729390480066991] |
|1    |1.0       |[0.14984516602550196,0.850154833974498]  |
|0    |0.0       |[0.644949035890669,0.355050964109331]    |
|0    |0.0       |[0.9136950493522981,0.08630495064770191] |
|0    |0.0       |[0.8846879853377492,0.11531201466225084] |
|1    |1.0       |[0.4826396805212696,0.5173603194787304]  |
|0    |1.0       |[0.29084458841296656,0.7091554115870334] |
|0    |0.0       |[0.9593691522270278,0.040630847772972094]|
|0    |1.0       |[0.33015591727015353,0.6698440827298464] |
|1    |1.0       |[0.14107916981588908,0.8589208301841109] |
|0    |0.0       |[0.7870901345432633,0.21290986545673674] |
|0    |0.0       |[0.810

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

metric = evaluator.evaluate(prediction)
metric_name = evaluator.getMetricName()

print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8636356373629883


In [11]:
model.write().overwrite().save(path='lr2')

In [12]:
from pyspark.ml import PipelineModel
m2 = PipelineModel.load(path='lr2')

In [13]:
prediction = m2.transform(df)
metric = evaluator.evaluate(prediction)
metric_name = evaluator.getMetricName()

print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8636356373629885
